#### This file is used for the testing of the data processing for a few articles instead of the whole parquet file

#### Instructions before running all the cells:
1. Change the input directory to the path that you stored the "merged_data_rag.parquet" and the respective output directory to your intended output folders
2. Install the required packages below

In [1]:
!pip install azure-cli


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [58]:
import glob
import json
import os

import pandas as pd
from azure.identity import AzureCliCredential, get_bearer_token_provider
from openai import AzureOpenAI

#### Creating a new parquet file with selected articles for testing

In [48]:
# Load the original Parquet file
input_file = "/Users/Richmond/Desktop/syn/healthhub-content-optimization/content-optimization/data/03_primary/merged_data.parquet/merged_data_rag.parquet"
df = pd.read_parquet(input_file)

# Filter the DataFrame to get the rows with specific article IDs
article_id_to_extract = [
    1442739,
    1445332,
    1439757,
    1439210,
    1443217,
    1435054,
    1437631,
    1434620,
    1444553,
    1445932,
    1437772,
    1435185,
    1435017,
    1464135,
    1445643,
    1434994,
    1435005,
    1434998,
    1435059,
]
filtered_df = df[df["id"].isin(article_id_to_extract)]

# Ensure the data_processing directory exists
output_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test"
os.makedirs(output_dir, exist_ok=True)

# Save the filtered DataFrame as a new Parquet file
output_file = os.path.join(output_dir, "index.parquet")
filtered_df.to_parquet(output_file, index=False)

print(f"Extracted articles {article_id_to_extract} and saved to {output_file}")

Extracted article [1442739, 1445332, 1439757, 1439210, 1443217, 1435054, 1437631, 1434620, 1444553, 1445932, 1437772, 1435185, 1435017, 1464135, 1445643, 1434994, 1435005, 1434998, 1435059] and saved to data_processing/index.parquet


#### Read the pre-processed parquet file

In [51]:
df = pd.read_parquet("/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/index.parquet")

In [52]:
df

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,has_image,related_sections,extracted_tables,extracted_raw_html_tables,extracted_links,extracted_headers,extracted_images,extracted_content_body,l1_mappings,l2_mappings
0,1435005,Enhancement for Active Seniors (EASE) by HDB,Enhancement for Active Seniors (EASE) by HDB,"Senior Health and Caregiving,Alerts and Adviso...",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/enhan...,enhancement_active_seniors,The HDB EASE programme aims to provide safer a...,"b'<div class=""ExternalClass26C3FCBE3D3D46728E8...",...,False,[],"[[[Singapore Citizen Households], [Flat Type, ...","[<table cellspacing=""0"" class=""ms-rteTable-def...","[[Housing Development Board (HDB), https://www...",[[Introduction to the Enhancement for Active S...,[],Introduction to the Enhancement for Active Sen...,None,None
1,1434994,Intermediate and Long-Term Care Services Subsi...,Intermediate and Long-Term Care Services Subsi...,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/subsi...,subsidies_intermediate_long_term_care,Those needing intermediate or long-term care f...,"b'<div class=""ExternalClassCD5BE21C38D64C5BB49...",...,False,[],[[[Revised Inpatient Sub-Acute and Rehabilitat...,"[<table align=""center"" border=""1"" cellpadding=...","[[Community Hospital Sub-Acute and Rehab, http...","[[, h2], [, h2], [What are Intermediate and Lo...",[],What are Intermediate and Long-Term Care Servi...,None,None
2,1435059,ElderShield,ElderShield,"Senior Health and Caregiving,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/elder...,eldershield,"Find out more about ElderShield, what it cover...","b'<div class=""ExternalClass99E003D691FF43D79DF...",...,True,"[Enhancement for Active Seniors (EASE), MediSa...",[[[ElderShield 300 (Default plan for people wh...,"[<table cellspacing=""0"" class=""ms-rteTable-def...","[[here, https://www.moh.gov.sg/careshieldlife]...","[[, h2], [Understanding ElderShield, h2], [Wha...","[[Activities of Elderly Daily Living, https://...",Understanding ElderShield\nElderShield is a se...,None,None
3,1434998,Interim Disability Assistance,Interim Disability Assistance,"Conditions and Illnesses,Senior Health and Car...",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/assis...,assistance_scheme_IDAPE,"Learn about IDAPE, a senior mobility fund for ...","b'<div class=""ExternalClass55CC36B715794B15844...",...,False,[Everything You Need to Know About ElderShield...,"[[[Household Monthly Income per Person, Monthl...","[<table class=""ms-rteTable-default table"" styl...",[[Everything You Need to Know About ElderShiel...,[[Interim Disability Assistance Programme for ...,[],Interim Disability Assistance Programme for th...,Support & Tools,Cost and Financing
4,1437631,Hyperlipidemia,Hyperlipidemia,"Conditions and Illnesses,Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/h...,hyperlipidemia,Hyperlipidemia which is also known as high blo...,"b'<div class=""ExternalClass522BD5542E734C6088A...",...,False,[],None,None,"[[Top 4 conditions of polyclinic attendance, h...","[[Introduction to Hyperlipidemia, h2], [What A...",[],Introduction to Hyperlipidemia\nHyperlipidemia...,Health Conditions,None
5,1437772,Heart Failure - Monitoring Weight and Blood Pr...,Heart Failure - Monitoring Weight and Blood Pr...,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.

#### Pass the table content into GPT-4o and create a new parquet with the new column "processed_table_content"

In [77]:
def ask(html_content: str) -> str:
    azure_credential = AzureCliCredential()
    token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")

    openai_client = AzureOpenAI(
        api_version="2024-03-01-preview",
        azure_endpoint="https://cog-zicuh2lqrbb2s.openai.azure.com",
        azure_ad_token_provider=token_provider,
    )

    # Updated prompt to discourage repetition
    prompt = """
    Below is the given full article html, extract the content of the tables, including any descriptions about the tables or may be helpful for the user to understand the tables. 
    Do not retain any of the text that are not helpful for the tables, remove all html tags and replace them with markdown format.
    Output the response as a single, readable string without any other sentences needed.

    {html_content}
    """

    # Prepare the messages for the API call
    query_messages = [
        {
            "role": "system",
            "content": "You are an AI assistant specialized in extracting structured content from HTML.",
        },
        {"role": "user", "content": prompt.format(html_content=html_content)},
    ]

    response = openai_client.chat.completions.create(
        messages=query_messages,
        model="chat",
        temperature=0.0,
        max_tokens=2000,
        n=1,
        seed=1234,
    )

    return response.choices[0].message.content


# Extract and process HTML tables


def process_html_tables(row):
    if row["has_table"]:
        return ask(row["content_body"])
    return None


# Apply processing to the DataFrame
df["processed_table_content"] = df.apply(process_html_tables, axis=1)

# Save to a new Parquet file
output_file = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/new_index.parquet"
df.to_parquet(output_file)

#### Read the post-processed parquet file

In [78]:
df1 = pd.read_parquet("/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/new_index.parquet")

#### Extract out the "content_body" as a .txt file to verify extraction

In [62]:
# Filter the DataFrame for rows where 'has_table' is True
filtered_df1 = df1[df1["has_table"]]

# Ensure the raw_content_body directory exists
raw_content_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/html_tables_cleaning"
os.makedirs(raw_content_dir, exist_ok=True)

# Process each article ID
for article_id in filtered_df1["id"].unique():
    # Filter the DataFrame for the current article ID
    article_df = filtered_df1[filtered_df1["id"] == article_id]

    # Extract the 'content_body' column
    processed_content = article_df["content_body"].tolist()

    # Define the output file path
    output_txt_file = f"{raw_content_dir}/raw_article_{article_id}.txt"

    # Write the content to a text file
    with open(output_txt_file, "w") as file:
        for content in processed_content:
            # Check if content is in bytes and decode if necessary
            if isinstance(content, bytes):
                content = content.decode("utf-8")
            file.write(content + "\n")  # Write each entry on a new line

    print(f"Raw content body for article {article_id} saved to {output_txt_file}")

Raw content body for article 1435005 saved to ./data_processing/raw_content_body/raw_article_1435005.txt
Raw content body for article 1434994 saved to ./data_processing/raw_content_body/raw_article_1434994.txt
Raw content body for article 1435059 saved to ./data_processing/raw_content_body/raw_article_1435059.txt
Raw content body for article 1434998 saved to ./data_processing/raw_content_body/raw_article_1434998.txt
Raw content body for article 1464135 saved to ./data_processing/raw_content_body/raw_article_1464135.txt
Raw content body for article 1445643 saved to ./data_processing/raw_content_body/raw_article_1445643.txt


#### Extract out the "processed_table_content" as a .txt file to verify extraction

In [69]:
# Filter the DataFrame for rows where 'has_table' is True
filtered_df = df1[df1["has_table"]]

# Ensure the raw_content_body directory exists
processed_content_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/html_tables_cleaning"
os.makedirs(processed_content_dir, exist_ok=True)

# Loop through each article ID to process
for article_id in article_id_to_extract:
    # Filter the DataFrame for the specific article ID
    article_df = filtered_df[filtered_df["id"] == article_id]

    if not article_df.empty:
        # Extract the 'processed_table_content' column
        processed_content = article_df["processed_table_content"].astype(str).tolist()

        # Define the output file path
        output_txt_file = f"{processed_content_dir}/processed_table_content_{article_id}.txt"

        # Write the content to a text file
        with open(output_txt_file, "w") as file:
            for content in processed_content:
                file.write(content + "\n")  # Write each entry on a new line

        print(f"Processed table content for article {article_id} saved to {output_txt_file}")

Processed table content for article 1464135 saved to ./data_processing/processed_table_content/processed_table_content_1464135.txt
Processed table content for article 1445643 saved to ./data_processing/processed_table_content/processed_table_content_1445643.txt
Processed table content for article 1434994 saved to ./data_processing/processed_table_content/processed_table_content_1434994.txt
Processed table content for article 1435005 saved to ./data_processing/processed_table_content/processed_table_content_1435005.txt
Processed table content for article 1434998 saved to ./data_processing/processed_table_content/processed_table_content_1434998.txt
Processed table content for article 1435059 saved to ./data_processing/processed_table_content/processed_table_content_1435059.txt


#### Check that processed_table_content is added as the last column of the new index

In [70]:
# Print all columns of the DataFrame
print("Columns in DataFrame:")
for column in df1.columns:
    print(column)

Columns in DataFrame:
id
content_name
title
article_category_names
cover_image_url
full_url
full_url2
friendly_url
category_description
content_body
keywords
feature_title
pr_name
alternate_image_text
date_modified
number_of_views
last_month_view_count
last_two_months_view
page_views
engagement_rate
bounce_rate
exit_rate
scroll_percentage
percentage_total_views
cumulative_percentage_total_views
content_category
to_remove
remove_type
has_table
has_image
related_sections
extracted_tables
extracted_raw_html_tables
extracted_links
extracted_headers
extracted_images
extracted_content_body
l1_mappings
l2_mappings
processed_table_content


#### Specify the columns to extract for article content and tables

In [71]:
columns_to_extract1 = [
    "id",
    "title",
    "cover_image_url",
    "full_url",
    "extracted_content_body",
    "content_category",
    "category_description",
    "has_table",  # Add this column to filter rows with tables after extracting the content
]

columns_to_extract2 = [
    "id",
    "title",
    "cover_image_url",
    "full_url",
    "processed_table_content",
    "content_category",
    "category_description",
]

#### Check the article count with tables and output their "full_url"

In [72]:
# List of article IDs to check
article_ids = [
    1442739,
    1445332,
    1439757,
    1439210,
    1443217,
    1435054,
    1437631,
    1434620,
    1444553,
    1445932,
    1437772,
    1435185,
    1435017,
    1464135,
    1445643,
    1434994,
    1435005,
    1434998,
    1435059,
]

# Filter the DataFrame for the rows with the specified article IDs
filtered_df = df[df["id"].isin(article_ids)]

# Check which rows have 'has_table' == True
articles_with_tables = filtered_df[filtered_df["has_table"]]

# Count of articles with tables
has_table_count = articles_with_tables.shape[0]

# Extract article IDs and full URLs
article_info = articles_with_tables[["id", "full_url"]]

# Adjust display settings for full URL visibility
pd.set_option("display.max_colwidth", None)

# Print the count and article info
print(f"Number of articles with 'has_table' == True: {has_table_count}")
print("Articles with tables:")
print(article_info)

# Optionally reset display settings if needed
pd.reset_option("display.max_colwidth")

Number of articles with 'has_table' == True: 6
Articles with tables:
        id  \
0  1435005   
1  1434994   
2  1435059   
3  1434998   
6  1464135   
7  1445643   

                                                                                 full_url  
0             https://www.healthhub.sg/a-z/costs-and-financing/enhancement_active_seniors  
1  https://www.healthhub.sg/a-z/costs-and-financing/subsidies_intermediate_long_term_care  
2                            https://www.healthhub.sg/a-z/costs-and-financing/eldershield  
3                https://www.healthhub.sg/a-z/costs-and-financing/assistance_scheme_IDAPE  
6           https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work  
7                                https://www.healthhub.sg/live-healthy/gettingsupportstis  


#### Specific articles extraction

In [76]:
# List of article IDs to extract
article_ids = [
    1442739,
    1445332,
    1439757,
    1439210,
    1443217,
    1435054,
    1437631,
    1434620,
    1444553,
    1445932,
    1437772,
    1435185,
    1435017,
    1464135,
    1445643,
    1434994,
    1435005,
    1434998,
    1435059,
]

# Specify the directory
output_directory = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/processed_articles"

# Ensure the directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop through each article ID in the list
for article_id in article_ids:
    # Extract the row based on the article ID
    row = df1[df1["id"] == article_id]

    # Check if row is empty
    if row.empty:
        print(f"Article ID {article_id} not found in the DataFrame.")
        continue

    row_number = row.index[0]  # Get the index of the row

    # Extract the specified columns for the given row
    extracted_row1 = df1.loc[row_number, columns_to_extract1]

    # Check if the row has a table
    has_table = extracted_row1["has_table"]

    # Convert the extracted row to a dictionary and remove 'has_table'
    extracted_data1 = extracted_row1.drop("has_table").to_dict()

    # Rename the key to "content"
    extracted_data1["content"] = str(extracted_data1.pop("extracted_content_body"))

    # Convert specific fields to strings and append the desired suffix
    extracted_data1["id"] = str(article_id) + "_content"

    # Wrap the dictionary in a list to match the desired format
    extracted_data_list1 = [extracted_data1]

    # Create unique file name using the article ID
    output_filename1 = f"{article_id}_content.json"

    # Define output path
    output_path1 = os.path.join(output_directory, output_filename1)

    # Export the extracted content body to a JSON file
    with open(output_path1, "w") as json_file:
        json.dump(extracted_data_list1, json_file, indent=4)

    # If there is a table, extract and save it as well
    if has_table:
        extracted_row2 = df1.loc[row_number, columns_to_extract2]
        extracted_data2 = extracted_row2.to_dict()
        extracted_data2["content"] = str(extracted_data2.pop("processed_table_content"))
        extracted_data2["id"] = str(article_id) + "_table"

        # Wrap the dictionary in a list to match the desired format
        extracted_data_list2 = [extracted_data2]

        # Create unique file name for the raw HTML tables
        output_filename2 = f"{article_id}_table.json"
        output_path2 = os.path.join(output_directory, output_filename2)

        # Export the extracted raw HTML tables to a JSON file
        with open(output_path2, "w") as json_file:
            json.dump(extracted_data_list2, json_file, indent=4)

        # Confirm the files were saved
        print(f"JSON files saved at: {output_path1} and {output_path2}")
    else:
        # Confirm the content body file was saved
        print(f"JSON file saved at: {output_path1}")

JSON file saved at: ./data_processing/processed_articles/1442739_content.json
JSON file saved at: ./data_processing/processed_articles/1445332_content.json
JSON file saved at: ./data_processing/processed_articles/1439757_content.json
JSON file saved at: ./data_processing/processed_articles/1439210_content.json
JSON file saved at: ./data_processing/processed_articles/1443217_content.json
JSON file saved at: ./data_processing/processed_articles/1435054_content.json
JSON file saved at: ./data_processing/processed_articles/1437631_content.json
JSON file saved at: ./data_processing/processed_articles/1434620_content.json
JSON file saved at: ./data_processing/processed_articles/1444553_content.json
JSON file saved at: ./data_processing/processed_articles/1445932_content.json
JSON file saved at: ./data_processing/processed_articles/1437772_content.json
JSON file saved at: ./data_processing/processed_articles/1435185_content.json
JSON file saved at: ./data_processing/processed_articles/1435017

#### Remove all .json files from a directory

In [83]:
def remove_json_files(directory):
    """
    Removes all .json files from the specified directory.

    Parameters:
    directory (str): The path to the directory where .json files are to be removed.
    """
    # Create the search pattern for .json files
    search_pattern = os.path.join(directory, "*.json")

    # Get a list of all .json files in the directory
    json_files = glob.glob(search_pattern)

    # Remove each .json file
    for file in json_files:
        os.remove(file)
        print(f"Removed file: {file}")


# Specify the directory where .json files are located
output_directory = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/processed_articles"

# Call the function to remove .json files
remove_json_files(output_directory)